In [8]:
import numpy as np
import cv2
import random
import pandas as pd
import time

import matplotlib.pyplot as plt
from seaborn import load_dataset

In [28]:
path= '../Alejandro_Proyectos/'
pathReferencia ='Imagenes Originales Gris/lena_gray.png'
pathRuido='lena/lena_gray_512_512_variance_25.png'
imagenReferencia = cv2.imread(path+pathReferencia)
imagenReferenciaGris = cv2.cvtColor(imagenReferencia, cv2.COLOR_RGB2GRAY)
imagenRuido = cv2.imread(path+pathRuido)
imagenRuidoGris = cv2.cvtColor(imagenRuido, cv2.COLOR_RGB2GRAY)
areaBusqueda = 21
tamañoParche = 8
split_punto = pathRuido.split(".")
split_guion = split_punto[0].split("_")
varianzaOriginal = int(split_guion[len(split_guion)-1])

# Transformada de Fourier

In [35]:
# def transformadaFourier(imagenGrises):
#     img1_Fourier = imagenGrises.astype('float')
#     #print(img1_Fourier,'Pixeles')
    
#     Fourier = np.fft.fft2(img1_Fourier)
#     #print(Fourier)
#     FourierShift = np.fft.fftshift(Fourier)
#     #print(FourierShift,'Fourier_Shift')

#     realFourierData = np.zeros((len(imagenGrises), len(imagenGrises)))

#     for i in range(len(realFourierData)):
#         for j in range(len(realFourierData)):
#             realFourierData[i,j] = FourierShift[i,j].real

#     return realFourierData

In [52]:
def transformadaFourier(imag):
    Nf = imag.shape[0]
    Nc = imag.shape[1]
    F1=np.arange(-Nf/2+1,Nf/2+1,1)
    F2=np.arange(-Nc/2+1,Nc/2+1,1)
    [X,Y]=np.meshgrid(F1,F2)
    #plt.plot(X,Y, marker='.', color='k', linestyle='none')
    D=np.sqrt(X**2+Y**2)
    D=D/np.max(D)
    #DEFINIR RADIO DE CORTE
    #Do=0.59 dependiendo el histograma de la imagen
    Do=.5
    #Creación del Filtro Ideal en 2D
    Huv=np.zeros((Nf,Nc))
    #PRIMERO CREAR EL FILTRO PASA BAJO IDEAL
    for i in range(Nf):
        for j in range(Nc):
            if(D[i,j]<Do):
                Huv[i,j]=1
#CONVERTIR A PASA ALTO IDEAL
#Huv=1-Huv
#----------------------------------------------------
#"FILTRO 2D PASA ALTO IDEAL"
#plt.imshow(np.uint8(255*Huv))
    imgD64 = np.float64(imag)
    Fourier = np.fft.fft2(imgD64)
    FourierShift = np.fft.fftshift(Fourier)
    Guv=Huv*FourierShift
    #MAGNITUD
    Guv_abs=np.abs(Guv)
    gxy=np.fft.ifft2(Guv_abs)
    gxy=np.abs(gxy)
    gxy=np.uint8(gxy)

    return gxy

In [53]:
transformadaFourier(imagenRuidoGris[0:0+8, 0: 0+8])

array([[186, 170, 159, 161, 157, 161, 159, 170],
       [171, 160, 162, 160, 156, 158, 155, 166],
       [157, 160, 162, 156, 157, 156, 150, 155],
       [153, 155, 155, 156, 160, 157, 153, 153],
       [148, 150, 153, 158, 161, 158, 153, 150],
       [153, 153, 153, 157, 160, 156, 155, 155],
       [157, 155, 150, 156, 157, 156, 162, 160],
       [171, 166, 155, 158, 156, 160, 162, 160]], dtype=uint8)

In [54]:
def algoritmoBJTF(posicionAltoImagenActual,posicionAnchoImagenActual,imgAct,imgRuido,areaBus,tamParche):
#     imagenActualFiltro = transformadaCosenoDiscreta(imgAct)
#     imagenRuidoFiltro = transformadaCosenoDiscreta(imgRuido)

    imagenActual = imgAct[posicionAltoImagenActual:posicionAltoImagenActual+tamParche, posicionAnchoImagenActual: posicionAnchoImagenActual+tamParche]
    imagenActualTF = transformadaFourier(imagenActual)
    dh1 = pd.DataFrame(columns=['ssd', 'coordenada_x', 'coordenada_y'])
    for m in range(posicionAltoImagenActual,posicionAltoImagenActual+areaBus-tamParche+1):
        for n in range(posicionAnchoImagenActual,posicionAnchoImagenActual+areaBus-tamParche+1):
            imagenReferencia = imgRuido[m:m+tamParche, n: n+tamParche]
            imagenReferenciaTF = transformadaFourier(imagenReferencia)
            lenParche=6 
            ssd=0
            for rx_Pache in range (2,lenParche):
                for ry_Pache in range (2,lenParche):
                    resta = int(imagenActualTF[rx_Pache,ry_Pache])-int(imagenReferenciaTF[rx_Pache,ry_Pache])
                    ssd += resta**2
#                     ssd += np.sum(np.power((imagenActualTCD[rx_Pache,ry_Pache]-imagenReferenciaTCD[rx_Pache,ry_Pache]),2))
            auxi = [ssd, m, n]
            dh2 = pd.DataFrame([auxi], columns=['ssd', 'coordenada_x', 'coordenada_y'])
            dh1= pd.concat([dh1, dh2], ignore_index=True)
#     print(posicionAltoImagenActual,posicionAnchoImagenActual)
    by_ascending_ssd = dh1.sort_values('ssd',ascending=True)
    by_ascending_ssd = by_ascending_ssd.reset_index(drop=True)
    by_ascending_ssd = by_ascending_ssd.astype(int)
    #by_ssd.head()
#     print(by_ascending_ssd)
    
    ##Aquí comienza la segunda etapa
    dhFefinitive = pd.DataFrame(columns=['ssd', 'coordenada_x', 'coordenada_y'])
    for iteraLista in range(0,8):
        Alto_x = by_ascending_ssd['coordenada_x'][iteraLista]
        Alto_y = by_ascending_ssd['coordenada_y'][iteraLista]
        imagenReferenciaDos = imgRuido[Alto_x:Alto_x+tamParche, Alto_y: Alto_y+tamParche]
        imagenReferenciaDosTF = transformadaFourier(imagenReferenciaDos)
        ssd=0
        lenParche = 7
        for i in range (2,lenParche):
            for j in range (2,lenParche):
                resta = int(imagenActualTF[i,j])-int(imagenReferenciaDosTF[i,j])
                ssd += resta**2
#                 ssd += np.sum(np.power((imagenActualTCD[i,j]-imagenReferenciaDosTCD[i,j]),2))
        auxi2 = [ssd, Alto_x, Alto_y]
        dhFefinitive2 = pd.DataFrame([auxi2], columns=['ssd', 'coordenada_x', 'coordenada_y'])
        dhFefinitive= pd.concat([dhFefinitive, dhFefinitive2], ignore_index=True)
    by_ascending_dhFefinitive = dhFefinitive.sort_values('ssd', ascending=True)
    by_ascending_dhFefinitive = by_ascending_dhFefinitive.reset_index(drop=True)
    by_ascending_dhFefinitive = by_ascending_dhFefinitive.astype(int)
#     print(by_ascending_dhFefinitive)
    posicionAltoImagenReferencia = by_ascending_dhFefinitive['coordenada_x'][0]
    posicionAnchoImagenReferencia = by_ascending_dhFefinitive['coordenada_y'][0]
    
    return posicionAltoImagenReferencia,posicionAnchoImagenReferencia

In [55]:
pxy_pref_BJTF = pd.DataFrame(columns=['coordenada_x', 'coordenada_y','referencia_x','referencia_y'])
altoImagen = 30 #imagenReferenciaGris.shape[0]
anchoImagen = 30 #imagenReferenciaGris.shape[1]

for posicionAlto in range(0,altoImagen-areaBusqueda): # altoImagen-areaBusqueda
    for posicionAncho in range(0,anchoImagen-areaBusqueda): # anchoImagen-areaBusqueda
        posicionAltoImagenReferencia,posicionAnchoImagenReferencia = algoritmoBJTF(posicionAlto,posicionAncho,imagenReferenciaGris,imagenRuidoGris,areaBusqueda,tamañoParche)
        auxi3 = [posicionAlto, posicionAncho, posicionAltoImagenReferencia,posicionAnchoImagenReferencia]
        pxy_pref_BJTF2 = pd.DataFrame([auxi3], columns=['coordenada_x', 'coordenada_y','referencia_x','referencia_y'])
        pxy_pref_BJTF = pd.concat([pxy_pref_BJTF, pxy_pref_BJTF2], ignore_index=True)
    
    
pxy_pref_BJTF
# imagen = algoritmoBJTF(imagenActual,areaBusqueda,tamañoParche)

,coordenada_x,coordenada_y,referencia_x,referencia_y
0,0,0,2,4
1,0,1,3,4
2,0,2,3,4
3,0,3,2,4
4,0,4,3,4
...,...,...,...,...
76,8,4,20,17
77,8,5,20,17
78,8,6,20,17
79,8,7,20,17


In [56]:
suma = 0
miu = 0
for i in range(0,len(pxy_pref_BJTF)-1):
    if pxy_pref_BJTF['coordenada_x'][i] == pxy_pref_BJTF['referencia_x'][i] and pxy_pref_BJTF['coordenada_y'][i] == pxy_pref_BJTF['referencia_y'][i]:
        suma += 1

miu =  suma/(len(pxy_pref_BJTF)-1)
miu
    

0.05